In [34]:
import pandas as pd
import numpy as np
import pickle
import regex as re
from underthesea import word_tokenize

In [70]:
df_news = pd.read_csv('data_news.csv')

In [65]:
df_news = df_news[df_news['label_id'] != 'label_id'].reset_index(drop=True)

In [66]:
df_labels = pd.DataFrame([
    {'label_id':'01', 'label': 'Đời Sống'},
    {'label_id':'02', 'label': 'Du Lịch'},
    {'label_id':'03', 'label': 'Giải Trí'},
    {'label_id':'04', 'label': 'Giáo Dục'},
    {'label_id':'05', 'label': 'Khoa Học'},
    {'label_id':'06', 'label': 'Kinh Doanh'},
    {'label_id':'07', 'label': 'Pháp Luật'},
    {'label_id':'08', 'label': 'Sức Khỏe'},
    {'label_id':'09', 'label': 'Thể Thao'}
])

In [67]:
df_news = df_news.merge(df_labels,on=['label_id'],how='left')

In [69]:
df_news[['title', 'content','label']].to_csv('data_news.csv',index=False)

## NLP

In [37]:
# Một số hàm tiền xử lý văn bản cần thiết
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

stopword = set()
file = r'vietnamese-stopwords.txt'
for line in open(file, "r",encoding="utf8"):
  line = line.strip('\n')
  stopword.add(line)

def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

def text_preprocess(document):
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = chuan_hoa_dau_cau_tieng_viet(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # xóa stopwords
    document = remove_stopwords(document)
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

In [71]:
df_news['title'] = df_news['title'].apply(lambda x : text_preprocess(x))
df_news['content'] = df_news['content'].apply(lambda x : text_preprocess(x))
df_news['text'] = df_news['title'] + ' ' + df_news['content']

In [72]:
df_news['label'].value_counts()

Du Lịch       630
Giáo Dục      623
Đời Sống      621
Kinh Doanh    620
Sức Khỏe      619
Pháp Luật     617
Thể Thao      616
Giải Trí      616
Khoa Học      608
Name: label, dtype: int64

In [73]:
df_news = df_news.reset_index(drop=True)

In [74]:
 # Thống kê các word xuất hiện ở tất cả các nhãn
total_label = 9
# Lấy túi từ vựng của dữ liệu đầu vào
vocab = {}
label_vocab = {}
for idx, row in  df_news.iterrows():
    words = row['text'].split()
    label = row['label']
    if label not in label_vocab:
        label_vocab[label] = {}
    for word in words:
        label_vocab[label][word] = label_vocab[label].get(word, 0) + 1
        if word not in vocab:
            vocab[word] = set()
        vocab[word].add(label)

In [75]:
# Chia tập train/test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

test_percent = 0.2

In [76]:
X_train, X_test, y_train, y_test = train_test_split(df_news['text'], df_news['label'], test_size=test_percent, random_state=42)
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
label_encoder.fit(y_test)
print(list(label_encoder.classes_), '\n')
Y_train = label_encoder.transform(y_train)
Y_test = label_encoder.transform(y_test)
Y_train = np_utils.to_categorical(Y_train, 9)
Y_test = np_utils.to_categorical(Y_test, 9)

['Du Lịch', 'Giáo Dục', 'Giải Trí', 'Khoa Học', 'Kinh Doanh', 'Pháp Luật', 'Sức Khỏe', 'Thể Thao', 'Đời Sống'] 



In [77]:
dict_label = {}
for idx, val in enumerate(list(label_encoder.classes_)):
  dict_label[str(idx)] = val
pickle.dump(dict_label,open('dict_label.pkl', 'wb'))

In [78]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

max_words = len(vocab.keys())

tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)
pickle.dump(tok,open('tokenize_text.pkl', 'wb'))

sequences = tok.texts_to_sequences(X_train)
max_len = 1600

sequences_matrix = pad_sequences(sequences,maxlen = max_len,padding='post')

print('Câu thứ nhất:', X_train[1])
print('Label câu thứ nhất:', y_train[1])
print('Độ dài: ', len(sequences[1]))
print('Tổng số câu', len(sequences))
print('Sequences_matrix shape: ', sequences_matrix.shape)
print('Sequences_matrix câu thứ nhất: ', sequences_matrix[1, :])

Câu thứ nhất: ngân_hàng quân_đội chủ_tịch quyết_định thay_đổi nhân_sự ngân_hàng quân_đội mb công_bố đồng_thời hội_đồng_quản_trị mb thông_qua thành_viên hội_đồng_quản_trị nhiệm_kỳ 2019 2024 11 10 thượng_tướng lê_hữu_đức chủ_tịch hội_đồng_quản_trị mb đánh_giá tổng công_trình_sư ngân_hàng dẫn_dắt mb 12 trải giai_đoạn chiến_lược dấu_ấn đậm_nét định_hướng chiến_lược xây_dựng nền_tảng quản_trị điều_hành lưu trung_thái sinh 1975 tốt_nghiệp thạc sỹ quản_trị kinh_doanh đại_học hawaii mỹ 26 gắn_bó mb mbs liền nắm vị_trí quan_trọng giám_đốc chi_nhánh giám_đốc nhân_sự phó tổng_giám_đốc thái bắt_đầu tham_gia hội_đồng_quản_trị mb 42013 trở_thành phó_chủ_tịch hội_đồng_quản_trị ngân_hàng 92013 giai_đoạn 2013 2017 tham_gia tái cấu_trúc công_ty thành_viên góp_phần xây_dựng hệ_sinh_thái mb thành tập_đoàn tài_chính đa_năng đầu 2017 bắt_đầu vị_trí ceo nhà_băng bổ_nhiệm vị_trí chủ_tịch hội_đồng_quản_trị mb thái đảm_nhận chức_danh ceo ngân_hàng thay phạm ánh phó tổng_giám_đốc giao phụ_trách ban điều_hành đảm

In [79]:
from keras.models import Sequential
from keras import layers
from keras.callbacks import EarlyStopping

embedding_dim =1000
num_filters = 128
kernel_size = 7
vocab_size = 49000

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=embedding_dim, 
                           input_length=1600))
model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
model.add(layers.Conv1D(64, kernel_size, activation='relu'))
model.add(layers.Conv1D(28, kernel_size, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(18, activation='relu'))
model.add(layers.Dense(9, activation='softmax'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

history = model.fit(sequences_matrix, Y_train, validation_split = 0.1,
batch_size=32, epochs=10, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1600, 1000)        49000000  
                                                                 
 conv1d (Conv1D)             (None, 1594, 128)         896128    
                                                                 
 conv1d_1 (Conv1D)           (None, 1588, 64)          57408     
                                                                 
 conv1d_2 (Conv1D)           (None, 1582, 28)          12572     
                                                                 
 global_max_pooling1d (Globa  (None, 28)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 18)                522       
                                                        

In [80]:
test_sequences = tok.texts_to_sequences(X_test)
max_len = 1600

test_sequences_matrix = pad_sequences(test_sequences,maxlen = max_len,padding='post')
score = model.evaluate(test_sequences_matrix, Y_test, verbose=0)
print(score)

y_predict = model.predict(test_sequences_matrix)

[0.19267897307872772, 0.8249551057815552]
35/35 [==============================] - 15s 415ms/step


In [81]:
i = 0
print(f"Câu test: {list(X_test)[i]}\nGiá trị dự đoán: {dict_label[str(np.argmax(y_predict[0]))]}")

Câu test: tết đồ_thừa tết 20 mẹ chồng khánh vân bỉm_sơn thanh_hóa gọi điện_thông_báo đụng lợn 80 kg ngăn mẹ nàng dâu dặn tùy tình_hình cân_đối hai bận bốn người_lớn hai đứa tết mẹ chốt đụng lợn vợ_chồng gửi 5 triệu đồng sắm tết vân _bận kinh_doanh trưa 30 tết vợ_chồng bố_mẹ lo tươm_tất kết_hôn vân phát hoảng thực_phẩm đồ_ăn khắp nia bếp bày hai chục bánh_chưng tủ_lạnh chất 5 giò_lụa bếp nồi thịt đông riu_riu lửa nồi cá kho xong sân giếng treo giò_xào hai xâu nem_chua treo lủng_lẳng mẹ mua lá nem khéo lắm sợ rách bảo tủ_lạnh rán thả rán sơ mẹ vân 50 nem _lượng thực_phẩm đồ nấu xong thắp hương vân phụ mẹ chuẩn_bị cơm_cúng tàn_hương gia_đình hạ mâm đồ_ăn nguội_ngắt hai bữa đầu đồ ngon chán thành_thử món đạm bánh_chưng cúng ế bữa bữa mâm cơm_mới hộp túi nồi đựng đồ ăn_thừa ngày_một tiêu_thụ phần_nào thậm_chí đồ ăn_thừa vân đi tiếc bố_mẹ đi thậm_chí đổ_lộn giò thịt_luộc thịt đông nồi bữa đun mùng 3 tết bụng_dạ hầu_như đình_công bố_mẹ uống men tiêu_hóa mẹ đi ruộng bẻ ngô luộc nhẹ_bụng vợ_chồ

In [58]:
model.save("model_cnn_news.h5")